In [2]:
import string
textRDD = sc.newAPIHadoopFile('../Data/Moby-Dick.txt',
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                               conf={'textinputformat.record.delimiter': "\r\n\r\n"}) \
            .map(lambda x: x[1])

sentences=textRDD.flatMap(lambda x: x.split(". "))

replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
RDD2= sentences.map(lambda sentence:sentence.encode('ascii').lower().replace('\r\n',' ').translate(replace_punctuation))

rdd7=RDD2.map(lambda x: x.split(' ')).map(lambda x: [e for e in x if e!=''])

def ngram(x,n):
    size=len(x)
    l=[]
    for i in range(size-n+1):
        l.append(x[i])
        for j in range(n-1):
            l[i]=l[i]+' '+x[i+j+1]
    return l

RDD13=rdd7.map(lambda x:ngram(x,3)).flatMap(lambda x:x).map(lambda ngram: (ngram,1))
RDD16=RDD13.reduceByKey(lambda x,y:x+y).map(lambda (c,v):(v,c)).sortByKey(False)
RDD16.take(10)

[(116, 'the sperm whale'),
 (109, 'of the whale'),
 (88, 'the white whale'),
 (64, 'one of the'),
 (60, 'of the sea'),
 (58, 'out of the'),
 (57, 'project gutenberg tm'),
 (53, 'part of the'),
 (53, 'the ship s'),
 (53, 'a sort of')]